<h3 align = "center"> Blockchain Techonologies </h3>
<h3 align = "center"> Practical 2 </h3>
<h3 align = "center"> Blockchain and Relay Attacks </h3>
<h5> 21BCE020 </h5>

---

##### Import libraries

In [1]:
import hashlib
import datetime

##### Block class to generate blocks

In [2]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash, nonce):
        self.index = index
        self.hash = hash
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.nonce = nonce

    def __repr__(self):
        return f"Block(Index: {self.index}, Hash: {self.hash}, Previous Hash: {self.previous_hash}, Timestamp: {self.timestamp}, Data: {self.data}, Nonce: {self.nonce})"

##### Chaining class to add blocks to the chain

In [3]:
class Blockchain:
    def __init__(self, difficulty=1):
        self.chain = [self.create_genesis_block()]
        self.difficulty = difficulty

    def create_genesis_block(self):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return Block(0, "0", timestamp, "Genesis Block", self.hash_block(0, "0", timestamp, "Genesis Block", 0), 0)

    def get_latest_block(self):
        return self.chain[-1]
    
    # Connect the blocks using previous hash value
    def add_block(self, data):
        previous_block = self.get_latest_block()
        index = previous_block.index + 1
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        previous_hash = previous_block.hash
        nonce, hash = self.proof_of_work(index, previous_hash, timestamp, data)
        new_block = Block(index, previous_hash, timestamp, data, hash, nonce)
        self.chain.append(new_block)
        
    # Compute hash for each block
    def hash_block(self, index, previous_hash, timestamp, data, nonce):
        value = f"{index}{previous_hash}{timestamp}{data}{nonce}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()
    
    # Compute nounce for each block based on difficulty
    def proof_of_work(self, index, previous_hash, timestamp, data):
        nonce = 0
        while True:
            hash = self.hash_block(index, previous_hash, timestamp, data, nonce)
            if hash[:self.difficulty] == '0' * self.difficulty:
                return nonce, hash
            nonce += 1
    
    # Check validity of the chain created
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            recalculated_hash = self.hash_block(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data, current_block.nonce)
            if current_block.hash != recalculated_hash:
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def tamper_block(self, index, new_data):
        if 0 < index < len(self.chain):
            self.chain[index].data = new_data
            self.recalculate_block_hash(index)
    
    # If blocks are tampered recalculate the hash 
    def recalculate_block_hash(self, index):
        current_block = self.chain[index]
        previous_hash = self.chain[index-1].hash if index > 0 else "0"
        nonce, hash = self.proof_of_work(current_block.index, previous_hash, current_block.timestamp, current_block.data)
        current_block.hash = hash
        current_block.nonce = nonce
        current_block.previous_hash = previous_hash

##### Execution

In [4]:
if __name__ == "__main__":
    blockchain = Blockchain(difficulty=1) 
    blockchain.add_block("Block 1")
    blockchain.add_block("Block 2")
    blockchain.add_block("Block 3")
    print("Initial Blockchain:")
    for block in blockchain.chain:
        print(block)
    print("\nBlockchain validity -> ", blockchain.is_chain_valid())
    blockchain.tamper_block(1, "Tampered Data")
    print("\nBlockchain post-tampering:")
    for block in blockchain.chain:
        print(block)
    print("\nBlockchain validity -> ", blockchain.is_chain_valid())

Initial Blockchain:
Block(Index: 0, Hash: 3e897775bab18aa9d6028a2310c66f2dd7286b24c582b8e7e44dc261c0d25ce2, Previous Hash: 0, Timestamp: 2024-09-30 17:38:19, Data: Genesis Block, Nonce: 0)
Block(Index: 1, Hash: 045177c084459b686d6e847ecb4f0c227a3f6e3ea4c112505f9fba50521215be, Previous Hash: 3e897775bab18aa9d6028a2310c66f2dd7286b24c582b8e7e44dc261c0d25ce2, Timestamp: 2024-09-30 17:38:19, Data: Block 1, Nonce: 36)
Block(Index: 2, Hash: 0082687209db2b4639df76f83111bf21ed7f85a2cf9ba0582bb1048e162a6c75, Previous Hash: 045177c084459b686d6e847ecb4f0c227a3f6e3ea4c112505f9fba50521215be, Timestamp: 2024-09-30 17:38:19, Data: Block 2, Nonce: 4)
Block(Index: 3, Hash: 010b42a1f1122fef9fe9ceea04b3826830c6447591eeedef174fde4a380fd86a, Previous Hash: 0082687209db2b4639df76f83111bf21ed7f85a2cf9ba0582bb1048e162a6c75, Timestamp: 2024-09-30 17:38:19, Data: Block 3, Nonce: 7)

Blockchain validity ->  True

Blockchain post-tampering:
Block(Index: 0, Hash: 3e897775bab18aa9d6028a2310c66f2dd7286b24c582b8e7e44d